In [1]:
# hide
import sys
sys.path.append("..")

In [2]:
# default_exp sweep_ensemble_utils

# Ensemble utils for WandB Sweeps

Per run, wandb stores pred_test and pred_valid pickle files containing:
* preds,targs,inp,losses,preds_denorm,targs_denorm,inp_denorm,pred_date

Per sweep, per run need to access these predictions to be ensembled. WandB API: 
* https://docs.wandb.com/library/reference/wandb_api
* https://docs.wandb.com/library/api
* https://docs.wandb.ai/library/public-api-guide
* https://docs.wandb.ai/ref/public-api

Need to ensemble these predictions calculate mean/median and variance as an uncertainty on the model prediction.

In [51]:
# export
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle
import pathlib
import wandb
from pathlib import Path

from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

from lib.read_data import *
from lib.stats_utils import cErrorMetrics, cStationary

In [4]:
# careful, they change things a lot...
#!pip3 install --upgrade wandb

In [5]:
!wandb --version

wandb, version 0.10.20


In [6]:
#!wandb login <add your key here>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Data from WandB API

In [4]:
# export
def fdownload_wandb_files(api, entity, project, sweep_ids, datadirs, wandbfname="preds_test_fname"):
    """
    download files with fname for all runs associated with list of sweep_ids into datadirs
    
    output:
    data[run_id] = [path, run.config]
    """
    data = {}
    
    for i in range(len(sweep_ids)):
        sweep = api.sweep("{}/{}/{}".format(entity, project, sweep_ids[i]))
        print("Sweep: ", sweep.config["name"])

        datadir = pathlib.Path(datadirs[i])
        datadir.mkdir(parents=True, exist_ok=True)

        for run in sweep.runs:
            run_id = run.id

            # fname on wandb api same for each run, need to rename with run_id
            fname_todownload = run.config[wandbfname]#.split("/")[1]
            fname_downloaded = "{}_{}.{}".format(fname_todownload.split(".")[0], run_id, fname_todownload.split(".")[1])

            # download (if doesn't already exist) and rename
            if not (datadir/fname_downloaded).is_file():
                file = run.file(fname_todownload)
                print("{} downloading {}...".format(run_id, fname_todownload))
                file.download(replace=False, root=datadir)
                (datadir/fname_todownload).rename(datadir/fname_downloaded)
            else:
                print("{} already downloaded.".format(fname_downloaded))
                
            # store location of file and run info
            data[run_id] = [datadir/fname_downloaded, run.config]
            
        return data

In [7]:
# export
def fget_pickled(sweeps, datadir, entity, project, pklname="models/preds_test.pickle", 
                 download=True, sweep_type="txt"):
    """
    download pickle files associated with runs in sweeps
    read data into dictionary first broken down by horizon, then indiviual runs
    
    sweep_type: "api" or "txt" [for manual .txt files with sweep run ids]
    sweeps == list of sweep ids or list of paths/filenames containing run ids
    """
    # path to download files to
    datadir = Path(datadir)
    datadir.mkdir(parents=True, exist_ok=True)

    api = wandb.Api()

    data = {}
    for sweep_i in sweeps:
        
        # case of wandb api working correctly:
        if sweep_type == "api":
            sweep = api.sweep("{}/{}/{}".format(entity, project, sweep_i))
            runs  = sweep.runs
            lrun  = len(runs)
            
        # case of txt files containing list of run_ids:
        elif sweep_type == "txt":
            df = pd.read_csv(sweep_i, delimiter=", ")
            run_ids = df["ID"].to_numpy().flatten()
            horizon = df["H"].to_numpy().flatten()
            lrun = len(run_ids)
        
        print(sweep_i, lrun)

        for i in range(lrun):
            
            if sweep_type == "api":
                run = runs[i]
                run_id = run.id
                h = run.config["horizon"]
                
            elif sweep_type == "txt":
                run_id = run_ids[i]
                # need access to run.config to get horizon (this is slow...)
                #run = api.run("{}/{}/{}".format(entity, project, run_id))
                h = horizon[i]

            # need to change name to include run_id or will overwrite
            fname_todownload = pklname
            fname_downloaded = "{}_{}.{}".format(fname_todownload.split(".")[0], run_id, fname_todownload.split(".")[1])

            # download pickle file (if doesn't already exist)
            if download:
                if not (datadir/fname_downloaded).is_file():
                    file = run.file(fname_todownload)
                    print("{} downloading {}...".format(run_id, fname_todownload))
                    file.download(replace=False, root=datadir)
                    (datadir/fname_todownload).rename(datadir/fname_downloaded)
                else:
                    print("{} already downloaded.".format(fname_downloaded))

            # read in pickle
            with open(datadir/fname_downloaded, 'rb') as handle:
                d = pickle.load(handle)

            # store by horizon, run_id
            try:
                data[h][run_id] = d
            except KeyError:
                data[h] = {}
                data[h][run_id] = d
            
    return data

In [8]:
entity  = "stardust-r"
project = "deep-learning-space-weather-forecasting"

# wandb sweep ids
#sweeps = ["inztzkl4", "607ppbji", "9ihcleuh", "2ajetr8i"]

# path to manual id text files
sweeps = ["./wandb_ensemble/ensemble/ensembleH3.txt", "./wandb_ensemble/ensemble/ensembleH5H7.txt", 
          "./wandb_ensemble/ensemble/ensembleH10H14.txt", "./wandb_ensemble/ensemble/ensembleH21H27.txt"]

# path to downloaded sweep run files
datadir = "../data/wandb_ensemble/ensemble"

In [9]:
%%time
data_valid = fget_pickled(sweeps, datadir, entity, project, pklname="models/preds_test.pickle", 
                          download=False, sweep_type="txt")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


./wandb_ensemble/ensemble/ensembleH3.txt 90
./wandb_ensemble/ensemble/ensembleH5H7.txt 180
./wandb_ensemble/ensemble/ensembleH10H14.txt 180
./wandb_ensemble/ensemble/ensembleH21H27.txt 180
CPU times: user 403 ms, sys: 2.66 s, total: 3.07 s
Wall time: 3.06 s


In [10]:
%%time
# for now this is not logged with wandb: must be manually generated by downloading model file and applying to training data
data_train = fget_pickled(sweeps, datadir, entity, project, pklname="models/preds_train.pickle", 
                          download=False, sweep_type="txt")

./wandb_ensemble/ensemble/ensembleH3.txt 90


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


./wandb_ensemble/ensemble/ensembleH5H7.txt 180
./wandb_ensemble/ensemble/ensembleH10H14.txt 180
./wandb_ensemble/ensemble/ensembleH21H27.txt 180
CPU times: user 2.39 s, sys: 12 s, total: 14.4 s
Wall time: 14.4 s


In [11]:
data_valid[3]["t8r3v9or"]

{'preds': array([[0.09384836, 0.09275597, 0.09365283],
        [0.09831063, 0.09721904, 0.09787896],
        [0.10078903, 0.09968247, 0.10018463],
        ...,
        [0.11686298, 0.11467203, 0.11406119],
        [0.11711943, 0.11463648, 0.1134624 ],
        [0.11596964, 0.11376598, 0.11256385]], dtype=float32),
 'targs': array([[0.10556812, 0.10712267, 0.1079706 ],
        [0.10712267, 0.1079706 , 0.10570944],
        [0.1079706 , 0.10570944, 0.10556812],
        ...,
        [0.12422272, 0.1240814 , 0.12323347],
        [0.1240814 , 0.12323347, 0.11899378],
        [0.12323347, 0.11899378, 0.11701526]]),
 'inp': array([[0.10825325, 0.10839457, 0.10924251, ..., 0.09821933, 0.09836066,
         0.1004805 ],
        [0.10839457, 0.10924251, 0.1079706 , ..., 0.09836066, 0.1004805 ,
         0.10556812],
        [0.10924251, 0.1079706 , 0.10811193, ..., 0.1004805 , 0.10556812,
         0.10712267],
        ...,
        [0.11644997, 0.116026  , 0.11517807, ..., 0.12210288, 0.12351611,
   

In [12]:
horizons = list(sorted(data_valid.keys()))
horizons

[3, 5, 7, 10, 14, 21, 27]

## Ensemble

In [52]:
# export
def fget_preds(subdict, limit_date=False, lcompdate=None, ucompdate=None):
    """
    stack predictions over ensemble runs
    get predictions and targets from runs
    shape: (90, 20729, 3) (n_ensemble_runs, n_windows, H)
    
    limit_date to crop for comparison with cls, esa etc.
    """
    # shorter lookbacks have more predictions
    # need to ensure all predictions are of same length and are combining the right set of predictions
    common_idx = []
    for run_id in subdict.keys():
        common_idx.append(len(subdict[run_id]["pred_date"]))
    idx = min(common_idx)

    # get common targets and prediction dates
    keys  = list(subdict.keys())
    targs = subdict[keys[0]]["targs_denorm"][-idx:]
    epoch = subdict[keys[0]]["pred_date"][-idx:]
    
    if limit_date:
        try:
            larg = np.argwhere(epoch==lcompdate)[0][0]
        except IndexError:
            larg = 0
        try:
            uarg = np.argwhere(epoch==ucompdate)[0][0] + 1
        except IndexError:
            uarg = -1
        targs = targs[larg:uarg]
        epoch = epoch[larg:uarg]
    
    # get predictions for each run
    preds = []
    for run_id in subdict.keys():
        if limit_date:
            preds.append(subdict[run_id]["preds_denorm"][-idx:][larg:uarg])
        else:
            preds.append(subdict[run_id]["preds_denorm"][-idx:])
    print(np.array(preds).shape, epoch[0], epoch[-1])
    
    return np.array(epoch), np.array(preds), np.array(targs)

In [53]:
# export
def fensemble(preds_valid, mode="mean"):
    """
    simple mean/median ensemble, std over ensemble
    """
    # reshape
    stack = np.dstack(preds_valid)
    
    if mode == "mean":
        ensemble_preds = np.mean(stack, axis=2)
    elif mode == "median":
        ensemble_preds = np.median(stack, axis=2)
  
    # variance in ensemble
    ensemble_std = np.std(stack, axis=2)
    #print(ensemble_std.shape)
        
    return ensemble_preds, ensemble_std

In [ ]:
def fensemble_linear_regression(preds_valid, targs_valid, preds_train, targs_train):
    """
    learn linear regression coefficients from training data (one for each day over horizon, H)
    apply to combine validation data
    
    X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
    # y = 1 * x_0 + 2 * x_1 + 3
    y = np.dot(X, np.array([1, 2])) + 3
    reg = LinearRegression().fit(X, y)
    reg.coef_ = array([1., 2.])
    reg.predict(np.array([[3, 5]])) = array([16.])
    """
    ensemble_preds = []
    
    H = preds_valid.shape[2]

    for h in range(H):
        # should be 90 (n_ensemble_runs) coefficients
        X = preds_train[:,:,h].T
        y = targs_train[:,h]
        reg = LinearRegression(fit_intercept=False).fit(X, y)

        X_valid = preds_valid[:,:,h].T
        y_valid = targs_valid[:,h]
        #print(reg.predict(X_valid), np.dot(X_valid, reg.coef_) + reg.intercept_, y_valid) # np.sum(np.multiply(X[-1], reg.coef_)) + intercept
        
        print(h, reg.score(X, y), reg.score(X_valid, y_valid))
        
        ensemble_preds.append(np.dot(X_valid, reg.coef_) + reg.intercept_)
        
    # not yet implemented with uncertainty

    return np.stack(ensemble_preds).T

In [54]:
# export
def fensemble_boosting_regressor(preds_valid, targs_valid, preds_train, targs_train, alpha=0.9):
    """
    Learn combination of ensemble members from training data using Gradient Boosting Regression
    Also provides prediction intervals (using quantile regression)
    alpha = % prediction interval
    
    https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_quantile.html
    https://towardsdatascience.com/how-to-generate-prediction-intervals-with-scikit-learn-and-python-ab3899f992ed
    """
    ensemble_preds = []
    ensemble_lower = []
    ensemble_upper = []
    
    H = preds_valid.shape[2]

    # run for each day over horizon
    for h in range(H):
        
        X_train = preds_train[:,:,h].T
        y_train = targs_train[:,h]
        X_test = preds_valid[:,:,h].T
        y_test = targs_valid[:,h]
        
        upper_model = GradientBoostingRegressor(loss="quantile", alpha=alpha)
        mid_model   = GradientBoostingRegressor(loss="ls")
        lower_model = GradientBoostingRegressor(loss="quantile", alpha=(1.0-alpha))                 

        # fit models
        lower_model.fit(X_train, y_train)
        mid_model.fit(X_train, y_train)
        upper_model.fit(X_train, y_train)
        
        # store predictions
        ensemble_preds.append(mid_model.predict(X_test))
        ensemble_lower.append(lower_model.predict(X_test))
        ensemble_upper.append(upper_model.predict(X_test))

    return np.stack(ensemble_preds).T, np.stack(ensemble_lower).T, np.stack(ensemble_upper).T

In [42]:
limit_date = False
lcompdate = None
ucompdate = None

predictions = {}

for horizon in horizons:
    print("Horizon: {}".format(horizon))
    
    # get training (if learning weights) and validation predictions and targets
    epoch_train, preds_train, targs_train = fget_preds(data_train[horizon])
    epoch_valid, preds_valid, targs_valid = fget_preds(data_valid[horizon], limit_date, lcompdate, ucompdate)
    
    ensemble_preds, ensemble_std = fensemble(preds_valid, mode="mean")
    print(ensemble_preds.shape, ensemble_std.shape)
    
    # ensemble and store predictions and uncertainty
    predictions[horizon] = [epoch_valid, targs_valid, ensemble_preds, ensemble_std]

Horizon: 3
(90, 20729, 3) 1950-01-19 2006-10-20
(90, 5183, 3) 2006-10-21 2020-12-28
(5183, 3) (5183, 3)
Horizon: 5
(90, 20718, 5) 1950-01-31 2006-10-21
(90, 5180, 5) 2006-10-22 2020-12-26
(5180, 5) (5180, 5)
Horizon: 7
(90, 20707, 7) 1950-02-12 2006-10-22
(90, 5177, 7) 2006-10-23 2020-12-24
(5177, 7) (5177, 7)
Horizon: 10
(90, 20690, 10) 1950-03-02 2006-10-23
(90, 5173, 10) 2006-10-24 2020-12-21
(5173, 10) (5173, 10)
Horizon: 14
(90, 20668, 14) 1950-03-26 2006-10-25
(90, 5167, 14) 2006-10-26 2020-12-17
(5167, 14) (5167, 14)
Horizon: 21
(90, 20628, 21) 1950-05-07 2006-10-27
(90, 5158, 21) 2006-10-28 2020-12-10
(5158, 21) (5158, 21)
Horizon: 27
(90, 20595, 27) 1950-06-12 2006-10-30
(90, 5149, 27) 2006-10-31 2020-12-04
(5149, 27) (5149, 27)


In [124]:
%%time

limit_date = False
lcompdate = None
ucompdate = None

mode = "mean" # "median", "boosting_regressor"

# read from pickled file (need for "boosting_regressor" as slow to train)
use_pickle = False
fname = "./wandb_ensemble/ensemble/ensemble_preds_{}_{}-{}.pickle".format(mode, lcompdate, ucompdate)

if use_pickle:
    try:
        with open(fname, 'rb') as handle:
            predictions = pickle.load(handle)
    except FileNotFoundError:
        print("Pickle file does not exist.")
        use_pickle = False

if not use_pickle:
    predictions = {}
    
    for horizon in horizons:
        print("Horizon: {}".format(horizon))

        # get training (if learning weights) and validation predictions and targets
        epoch_train, preds_train, targs_train = fget_preds(data_train[horizon])
        epoch_valid, preds_valid, targs_valid = fget_preds(data_valid[horizon], limit_date, lcompdate, ucompdate)

        # ensemble and store predictions and uncertainty or prediction interval
        if mode == "boosting_regressor"
            ensemble_preds, ensemble_lower, ensemble_upper = fensemble_boosting_regressor(preds_valid, targs_valid, 
                                                                                          preds_train, targs_train, 
                                                                                          alpha=0.95)
            predictions[horizon] = [epoch_valid, targs_valid, ensemble_preds, ensemble_lower, ensemble_upper]
        else:
            ensemble_preds, ensemble_std = fensemble(preds_valid, mode=mode)
            predictions[horizon] = [epoch_valid, targs_valid, ensemble_preds, ensemble_std]

    # if pickle does not already exist, generate
    if not Path(fname).is_file():
        with open(fname, 'wb') as handle:
            pickle.dump(predictions, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
predictions

ensemble_preds_mean_None-None.pickle
Pickle file does not exist.
CPU times: user 0 ns, sys: 1.45 ms, total: 1.45 ms
Wall time: 828 µs


{}

## Metrics

In [63]:
# export
def fmse(y_true, y_pred):
    """
    Mean square error.
    (from sklearn.metrics import mean_squared_error)
    """
    # return np.sum( (np.array(y_true) - np.array(y_pred))**2 ) / len(y_true)
    return mean_squared_error(y_true, y_pred)

def fmape(y_true, y_pred): 
    """
    mean_absolute_percentage_error.
    (can cause division-by-zero errors)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def fcc_pearsonr(y_true, y_pred):
    """
    pearson correlation coefficient
    """
    try:
        return pearsonr(y_true, y_pred)
    except TypeError:
        return pearsonr(y_true.flatten(), y_pred.flatten())
    
def frmse(y_true, y_pred):
    """
    Root mean square error.
    (from sklearn.metrics import mean_squared_error)
    """
    return np.sqrt(mean_squared_error(y_true, y_pred))

def fmae(y_true, y_pred):
    """
    mean absolute error
    """
    #return np.sum( np.abs(np.array(y_true) - np.array(y_pred)) ) / len(y_true)
    return mean_absolute_error(y_true, y_pred)

def fme(y_true, y_pred):
    """
    bias
    see Liemohn 2018 (model - observed)
    """
    return np.sum( np.array(y_pred) - np.array(y_true) ) / len(y_true)

In [61]:
# export
def fmse_std(y_true, y_pred):
    """
    Mean square error.
    (from sklearn.metrics import mean_squared_error)
    There mse stds are massive: would need to plot using rmse
    """
    se = (np.array(y_true) - np.array(y_pred))**2
    mse = np.mean(se)#, axis=0
    sdse = np.std(se)#, axis=0
    return sdse
    
def fmape_std(y_true, y_pred): 
    """
    mean_absolute_percentage_error.
    (can cause division-by-zero errors)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    ape = np.abs((y_true - y_pred) / y_true)
    mape = np.mean(ape) * 100
    sdape = np.std(ape) * 100
    return sdape

def fcc_pearsonr_std(y_true, y_pred):
    """
    pearson correlation coefficient
    """
    return np.nan

def frmse_std(y_true, y_pred):
    """
    Root mean square error.
    (from sklearn.metrics import mean_squared_error)
    """
    se = (np.array(y_true) - np.array(y_pred))**2
    mse = np.mean(se)#, axis=0
    sdse = np.std(se)#, axis=0
    return np.sqrt(sdse)

def fmae_std(y_true, y_pred):
    """
    mean absolute error
    """
    ae = np.abs(np.array(y_true) - np.array(y_pred))
    mae = np.mean(ae)#, axis=0
    sdae = np.std(ae)#, axis=0
    return sdae

def fme_std(y_true, y_pred):
    """
    bias
    see Liemohn 2018 (model - observed)
    """
    e = np.array(y_pred) - np.array(y_true)
    me = np.mean(e)#, axis=0
    sde = np.std(e)#, axis=0
    return sde

In [62]:
# export
def fget_metrics(y_pred, y_true):
    return [fmse(y_true, y_pred), fmape(y_true, y_pred), fcc_pearsonr(y_true, y_pred)[0], 
            frmse(y_true, y_pred), fmae(y_true, y_pred), fme(y_true, y_pred)]

def fget_metrics_std(y_pred, y_true):
    return [fmse_std(y_true, y_pred), fmape_std(y_true, y_pred), fcc_pearsonr_std(y_true, y_pred), 
            frmse_std(y_true, y_pred), fmae_std(y_true, y_pred), fme_std(y_true, y_pred)]

## Compare to external data

In [ ]:
# export
def fget_external_forecasts(config):
    """
    generate dataframe containing forecasts and "truths" for external sources
    get persistence 
    
    test:
    #config = AttrDict()
    #config.update(user_config)
    #fget_external_forecasts(config)
    """
    # ESA
    if "esa" in config.data_comp:
        dataobj_esa = cESA_SWE()
        # read in archive data
        df = dataobj_esa.fget_data(filenames=config.esa_archive_fname)[config.esa_archive_key]
        df.set_index('ds', inplace=True)
        udate_a = (dt.datetime.strptime(config.date_ulim, "%Y-%m-%d") + dt.timedelta(days=27)).strftime("%Y-%m-%d")
        df_lim = df[config.date_llim:udate_a]
        #dfa_esa = dataobj_esa.finterpolate(df_lim, config.interp_freq)
        df_daily = dataobj_esa.fget_daily(df_lim, config.get_daily_method)
        dfa_esa  = dataobj_esa.fmissing_data(df_daily, config.missing_data_method)
        # read in esa forecast data  
        dff_esa = dataobj_esa.fget_data(filenames=config.esa_forecast_fname)[config.esa_forecast_key]
        # add "truth" from archive to forecast
        dff_comp_esa = dataobj_esa.fget_forecast_comp(dff_esa, dfa_esa, cname="y")
        
        # rename columns
        #dfa_esa.columns = ['gendate' , 'ds', 'y_esa_true'] 
        dff_comp_esa.columns = ['gendate' , 'ds', 'y_esa', 'y_esa_true']        
        # Calculate persistence
        #dff_comp_esa = dataobj_esa.fget_persistence(dfa_esa, 'y_esa_true', "persistence_esa")
        dff_comp_esa = dataobj_esa.fget_persistence(dff_comp_esa, 'y_esa_true', "persistence_esa")        
    
    # CLS-CNES
    if "cls" in config.data_comp:
        dataobj_cls = cCLS_CNES()
        # read in archive data and restrict to ds and key variable
        # need to ensure upper date for archive is 30 days ahead of upper date of forecast gendate
        udate_a = (dt.datetime.strptime(config.cls_forecast_udate, "%Y-%m-%d") + dt.timedelta(days=30)).strftime("%Y-%m-%d")
        dfa_cls = dataobj_cls.fget_archive_data(config.cls_datadir, config.cls_forecast_ldate, udate_a)
        dfa_cls = dfa_cls[['ds',config.cls_key]]
        dfa_cls = dfa_cls.set_index("ds")
        # read in forecast data and restrict to key variable
        dff_cls = dataobj_cls.fget_forecast_data(config.cls_datadir, config.cls_forecast_ldate, config.cls_forecast_udate)

        dff_cls = dff_cls[['gendate', 'ds', "{}_c".format(config.cls_key)]]
        # add "truth" from archive to forecast
        dff_comp_cls = dataobj_cls.fget_forecast_comp(dff_cls, dfa_cls, cname=config.cls_key)
        
        # rename columns
        dff_comp_cls.columns = ['gendate' , 'ds', 'y_cls', 'y_cls_true']  
        # Calculate persistence
        dff_comp_cls = dataobj_cls.fget_persistence(dff_comp_cls, 'y_cls_true', "persistence_cls")
        
        # botch
        # include esa persistence in cls only table
        if "esa" not in config.data_comp:
            dataobj_esa = cESA_SWE()
            # read in archive data
            df = dataobj_esa.fget_data(filenames=config.esa_archive_fname)[config.esa_archive_key]
            df.set_index('ds', inplace=True)

            ldate = (dt.datetime.strptime(config.cls_forecast_ldate, "%Y-%m-%d") - dt.timedelta(days=1)).strftime("%Y-%m-%d")
            df_lim = df[ldate:udate_a] # hard code as takes 1 day later for some reason
            #dfa_esa = dataobj_esa.finterpolate(df_lim, config.interp_freq)
            df_daily = dataobj_esa.fget_daily(df_lim, config.get_daily_method)
            dfa_esa  = dataobj_esa.fmissing_data(df_daily, config.missing_data_method)
            # add "truth" from archive to forecast
            dff_comp_cls1 = dataobj_cls.fget_forecast_comp(dff_cls, dfa_esa, cname="y_cls")

            dff_comp_cls1.columns = ['gendate' , 'ds', 'y_cls', 'y_esa_true']

            # drop gendates that don't give full forcast (depends on horizon)
            dff_comp_cls1 = dff_comp_cls1[dff_comp_cls1.groupby('gendate').gendate.transform('count')>=27].copy()

            # Calculate persistence
            dff_comp_cls1 = dataobj_cls.fget_persistence(dff_comp_cls1, 'y_esa_true', "persistence_esa")

            dff_comp_cls = pd.merge(dff_comp_cls, dff_comp_cls1, on=['gendate', 'ds', 'y_cls'])
            
    
    # COMBINE AND RETURN
    if ("esa" in config.data_comp) and ("cls" in config.data_comp):
        df_comp = pd.merge(dff_comp_esa, dff_comp_cls, on=['gendate', 'ds'])
        return df_comp
    elif "esa" in config.data_comp:
        return dff_comp_esa
    elif "cls" in config.data_comp:
        return dff_comp_cls

In [ ]:
# export
def fget_external_metrics(dff, config):
    metrics = {}
    metrics_std = {}
    
    if "esa" in config.data_comp:
        metrics["ESA"] = fget_metrics(dff.y_esa, dff.y_esa_true)
        metrics_std["ESA"] = fget_metrics_std(dff.y_esa, dff.y_esa_true)
        
    if "cls" in config.data_comp:
        metrics["CLS"] = fget_metrics(dff.y_cls, dff.y_cls_true)
        metrics_std["CLS"] = fget_metrics_std(dff.y_cls, dff.y_cls_true)
    
    metrics["PERSISTENCE"] = fget_metrics(dff.persistence_esa, dff.y_esa_true)
    metrics_std["PERSISTENCE"] = fget_metrics_std(dff.persistence_esa, dff.y_esa_true)
    #metrics["PERSISTENCE"] = fget_metrics(dff.persistence_cls, dff.y_cls_true)
    
    return metrics, metrics_std

## Export

In [1]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_stats_utils.ipynb.
Converted 02_plot_utils.ipynb.
Converted 03_read_data.ipynb.
Converted 04_model.ipynb.
Converted 05_solar_flux_time_series_forecasting.ipynb.
Converted 06_ensemble_utils.ipynb.
Converted index.ipynb.
